In [16]:
import xlwings as xw
import pandas as pd
import matplotlib.pyplot as plt

# Constants
FILE_NAME = 'shearwall_data_backup.xlsx'
SHEET_NAME = 'SW_info'
LONGEST_DIM_NS_CELL = 'J10'
LONGEST_DIM_EW_CELL = 'J11'

# Functions
def extract_wall_data(sheet, start_col, id_col, length_col, height_col, x_col, y_col):
    """Extracts wall data from Excel and returns lists of attributes."""
    last_row = sheet.range(f'{start_col}{sheet.cells.last_cell.row}').end('up').row
    num_walls = int(sheet.range(f'{start_col}{last_row}').value)
    start_row = last_row - num_walls

    walls, lengths, heights, x_list, y_list = [], [], [], [], []
    for row in range(start_row + 1, start_row + 1 + num_walls):
        wall = {
            'id': sheet.range(f'{id_col}{row}').value,
            'length': sheet.range(f'{length_col}{row}').value,
            'height': sheet.range(f'{height_col}{row}').value,
            'x': sheet.range(f'{x_col}{row}').value,
            'y': sheet.range(f'{y_col}{row}').value,
        }
        walls.append(wall)
        lengths.append(wall['length'])
        heights.append(wall['height'])
        x_list.append(wall['x'])
        y_list.append(wall['y'])
    return walls, lengths, heights, x_list, y_list

def calculate_center_of_mass(lengths, coords):
    """Calculates the center of mass for a set of walls."""
    total_length = sum(lengths)
    com = sum(length * coord for length, coord in zip(lengths, coords)) / total_length
    return com

def calculate_normalized_lengths(lengths):
    """Calculates normalized lengths."""
    total_length = sum(lengths)
    return [length / total_length for length in lengths]

def calculate_m_torsion(lengths, longest_dimension):
    """Calculates torsion values for walls."""
    return [0.1 * longest_dimension for _ in lengths]

def calculate_rigidity(lengths, coords, com):
    """Calculates rigidity-related values for walls."""
    d_list = [abs(com - coord) for coord in coords]
    kdx_list = [length * d**2 for length, d in zip(lengths, d_list)]
    Jp = sum(kdx_list)
    return d_list, Jp

def calculate_v_total(normalized_lengths, torsion_list, k_list, Jp):
    """Calculates the total shear distribution for walls."""
    return [
        norm_length + (tor * k) / Jp
        for norm_length, tor, k in zip(normalized_lengths, torsion_list, k_list)
    ]

def plot_shear_wall_layout(ax, x_list, y_list, lengths, color, label_prefix):
    """Plots shear walls in a given direction."""
    for i, (x, y, length) in enumerate(zip(x_list, y_list, lengths)):
        if color == 'blue':  # Vertical (north-south)
            ax.plot([x, x], [y, y + length], label=f"{label_prefix} Wall {i+1}", color=color)
        else:  # Horizontal (east-west)
            ax.plot([x, x + length], [y, y], label=f"{label_prefix} Wall {i+1}", color=color)

# Workbook and Sheet Setup
wb = xw.Book(FILE_NAME)
sheet = wb.sheets[SHEET_NAME]
ws = sheet

# Extract Data for North-South and East-West Walls
walls_ns, lengths_ns, heights_ns, x_ns, y_ns = extract_wall_data(sheet, 'D', 'C', 'E', 'F', 'G', 'H')
walls_ew, lengths_ew, heights_ew, x_ew, y_ew = extract_wall_data(sheet, 'P', 'O', 'Q', 'R', 'S', 'T')

# Calculate Centers of Mass
com_x_ns = calculate_center_of_mass(lengths_ns, x_ns)
com_y_ew = calculate_center_of_mass(lengths_ew, y_ew)

# Output CoM Results to Excel
sheet.range('J13').value = com_x_ns
sheet.range('J14').value = com_y_ew

# Calculate Normalized Lengths
norm_lengths_ns = calculate_normalized_lengths(lengths_ns)
norm_lengths_ew = calculate_normalized_lengths(lengths_ew)

# Output Normalized Lengths to Excel
sheet.range('J29').options(transpose=True).value = norm_lengths_ns
sheet.range('V29').options(transpose=True).value = norm_lengths_ew

# Calculate Torsion and Rigidity Values
longest_dim_ns = sheet.range(LONGEST_DIM_NS_CELL).value
longest_dim_ew = sheet.range(LONGEST_DIM_EW_CELL).value

m_tor_ns = calculate_m_torsion(lengths_ns, longest_dim_ns)
m_tor_ew = calculate_m_torsion(lengths_ew, longest_dim_ew)

sheet.range('K29').options(transpose=True).value = m_tor_ns
sheet.range('W29').options(transpose=True).value = m_tor_ew

d_x_ns, Jpx = calculate_rigidity(lengths_ns, x_ns, com_x_ns)
d_y_ew, Jpy = calculate_rigidity(lengths_ew, y_ew, com_y_ew)

sheet.range('I29').options(transpose=True).value = d_x_ns
sheet.range('U29').options(transpose=True).value = d_y_ew
sheet.range('J19').value = Jpx
sheet.range('J20').value = Jpy

# Calculate kdy for EW walls
kdy_list = [length * y for length, y in zip(lengths_ew, y_ew)]

# Calculate V_total for NS and EW directions
V_total_ns_list = calculate_v_total(norm_lengths_ns, m_tor_ns, d_x_ns, Jpx)
V_total_ew_list = calculate_v_total(norm_lengths_ew, m_tor_ew, kdy_list, Jpy)

# Output V_total Results to Excel
sheet.range('L29').options(transpose=True).value = V_total_ns_list
sheet.range('X29').options(transpose=True).value = V_total_ew_list

# Visualization
fig, ax = plt.subplots(figsize=(8, 6))
plot_shear_wall_layout(ax, x_ns, y_ns, lengths_ns, color='blue', label_prefix='NS')
plot_shear_wall_layout(ax, x_ew, y_ew, lengths_ew, color='red', label_prefix='EW')
ax.scatter(com_x_ns, com_y_ew, color='green', s=100, label='(CoM_x_NS, CoM_y_EW)')
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')
ax.set_title('Shear Wall Layout')
# ax.legend(loc='upper right')
ax.grid(True)
ax.axis('equal')

# Embed the plot into Excel
ws.pictures.add(fig, name='ShearWallPlot', update=True, anchor=ws.range('C64'))
plt.show()
